# 📡 2.45 GHz Patch Antenna 設計工具
此工具可根據輸入的天線參數計算 Patch 天線的關鍵尺寸與效能估算，並繪製結構圖。

In [ ]:
# 📦 安裝必要套件
!pip install ipywidgets

In [ ]:
# 📌 匯入模組
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# 🔧 設定輸入參數 GUI
f_r_input = widgets.FloatText(value=2.45, description='頻率(GHz):')
er_input = widgets.FloatText(value=4.4, description='εr:')
h_input = widgets.FloatText(value=1.6, description='厚度 h(mm):')
t_input = widgets.FloatText(value=0.034, description='銅箔 t(mm):')

ui = widgets.VBox([f_r_input, er_input, h_input, t_input])
display(ui)

In [ ]:
# 🧮 Patch 天線計算邏輯
def patch_design(f_r, er, h, t):
    c = 3e8
    f_r = f_r * 1e9
    h = h / 1000
    t = t / 1000

    W = c / (2 * f_r * np.sqrt((er + 1) / 2))
    er_eff = (er + 1) / 2 + (er - 1) / 2 * (1 / np.sqrt(1 + 12 * h / W))

    delta_L = h * 0.412 * ((er_eff + 0.3) * (W / h + 0.264)) / ((er_eff - 0.258) * (W / h + 0.8))
    L_eff = c / (2 * f_r * np.sqrt(er_eff))
    L = L_eff - 2 * delta_L

    Q = (1 / (np.sqrt(er_eff) - 1)) * (L / h) * (1 + (1 / (W / h)))
    BW_frac = 1.5 / Q
    BW = BW_frac * f_r / 1e6

    return W * 1000, er_eff, L_eff * 1000, L * 1000, BW

In [ ]:
# 📐 畫出 Patch 結構圖
def draw_patch(W, L):
    fig, ax = plt.subplots()
    ax.add_patch(plt.Rectangle((0, 0), W, L, edgecolor='black', facecolor='lightblue'))
    ax.text(W/2, L/2, 'Patch', ha='center', va='center')
    ax.set_xlim(-5, W + 5)
    ax.set_ylim(-5, L + 5)
    ax.set_aspect('equal')
    ax.set_title('Patch Antenna 結構圖')
    plt.xlabel('寬度 W (mm)')
    plt.ylabel('長度 L (mm)')
    plt.grid(True)
    plt.show()

In [ ]:
# ▶️ 按鈕計算與顯示結果
output = widgets.Output()

def on_calculate_clicked(b):
    with output:
        output.clear_output()
        W, er_eff, L_eff, L, BW = patch_design(f_r_input.value, er_input.value, h_input.value, t_input.value)
        print(f'貼片寬度 W = {W:.2f} mm')
        print(f'有效介電常數 ε_eff = {er_eff:.4f}')
        print(f'貼片有效長度 L_eff = {L_eff:.2f} mm')
        print(f'貼片實際長度 L = {L:.2f} mm')
        print(f'頻寬估算 BW ≈ {BW:.2f} MHz')
        draw_patch(W, L)

button = widgets.Button(description='開始計算')
button.on_click(on_calculate_clicked)

display(button, output)